## Тестовые случаи

Входные данные представлены парой документов в формате `doc`

1. Документы с текстом

- Два идентичных файла
- Два идентичных по содержанию с разным форматированием
- 1 файл целиком состоит из части текста взятого из файла 2
- 1 файл целиком состоит из части текста взятого из файла 2 с другим форматированием
- 1 файл содержит текст из файла 2
- 1 файл содержит текст из файла 2 c другим форматированием
- 1 файл содержит перефразированный текст из файла 2

1. Документы с изображениями

- Два идентичных файла
- 1 файл целиком состоит из части картинок из файла 2
- 1 файл содержит картинки из файла 2
- 1 файл содержит ресайзнутые или смещенные картинки из файла 2

3. Документы с текстом и с изображениями

- Два идентичных файла
- 1 файл содержит текст и картинки из файла 2


## Наивные тесты с документами

Идентичные файлы

Для идентичных файлов достаточно сравнить их хэши, чтобы определить, что один из файлов является дубликатом.

In [1]:
import hashlib

def sha256_hash_file(filename: str) -> str:
    sha256_hash = hashlib.sha256()

    with open(filename,"rb") as f:
        for byte_block in iter(lambda: f.read(4096),b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()


In [2]:
hash1 = sha256_hash_file("./data/text-report/case_1_input_1.docx")
hash2 = sha256_hash_file("./data/text-report/case_1_input_2.docx")

hash1 == hash2

True

Идентичные файлы с разным форматированием

Сравнение по хэшу одинаковых по содержанию, но разных по форматирированию файлов бесполезно.

In [3]:
hash1 = sha256_hash_file("./data/text-report/case_2_input_1.docx")
hash2 = sha256_hash_file("./data/text-report/case_2_input_2.docx")

hash1 == hash2

False

Попробуем вытащить из файла весь текст и сравнить его

In [4]:
# code snippet from <https://github.com/ankushshah89/python-docx2txt/blob/master/docx2txt/docx2txt.py>

import xml.etree.ElementTree as ET
import zipfile
import os
import re
from dataclasses import dataclass

@dataclass
class InDocImage:
    fname: str
    data: str

class Docx2Txt:
    _nsmap = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

    def __init__(self, docx: any) -> None:
        self._docx = docx

        self._text = ""
        self._images: list[InDocImage] = []

        self._process()

    def get_text(self) -> str:
        return self._text

    def get_images(self) -> list[InDocImage]:
        return self._images

    def _qn(self, tag):
        """
        Stands for 'qualified name', a utility function to turn a namespace
        prefixed tag name into a Clark-notation qualified tag name for lxml. For
        example, ``qn('p:cSld')`` returns ``'{http://schemas.../main}cSld'``.
        Source: https://github.com/python-openxml/python-docx/
        """
        prefix, tagroot = tag.split(':')
        uri = self._nsmap[prefix]
        return '{{{}}}{}'.format(uri, tagroot)

    def _xml2text(self, xml):
        """
        A string representing the textual content of this run, with content
        child elements like ``<w:tab/>`` translated to their Python
        equivalent.
        Adapted from: https://github.com/python-openxml/python-docx/
        """
        text = u''
        root = ET.fromstring(xml)
        for child in root.iter():
            if child.tag == self._qn('w:t'):
                t_text = child.text
                text += t_text if t_text is not None else ''
            elif child.tag == self._qn('w:tab'):
                text += '\t'
            elif child.tag in (self._qn('w:br'), self._qn('w:cr')):
                text += '\n'
            elif child.tag == self._qn("w:p"):
                text += '\n\n'
        return text


    # TODO: extract other types of embedded files
    # TODO: extract tables
    # TODO: header and footer processing
    def _process(
            self,
            add_header=False,
            add_footer=False,
    ):
        text = u''

        # unzip the docx in memory
        with zipfile.ZipFile(self._docx) as zipf:
            filelist = zipf.namelist()

            if add_header:
                # get header text
                # there can be 3 header files in the zip
                header_xmls = 'word/header[0-9]*.xml'
                for fname in filelist:
                    if re.match(header_xmls, fname):
                        text += self._xml2text(zipf.read(fname))

            # get main text
            doc_xml = 'word/document.xml'
            text += self._xml2text(zipf.read(doc_xml))

            if add_footer:
                # get footer text
                # there can be 3 footer files in the zip
                footer_xmls = 'word/footer[0-9]*.xml'
                for fname in filelist:
                    if re.match(footer_xmls, fname):
                        print(zipf.read(fname))
                        text += self._xml2text(zipf.read(fname))

            self._text = text

            for fname in filelist:
                _, extension = os.path.splitext(fname)
                if extension in [".jpg", ".jpeg", ".png", ".bmp"]:
                    self._images.append(InDocImage(**{
                        "fname": os.path.basename(fname),
                        "data": zipf.read(fname),   
                    }))


In [5]:
text1 = Docx2Txt("./data/text-report/case_2_input_1.docx").get_text()
text2 = Docx2Txt("./data/text-report/case_2_input_2.docx").get_text()

hasher1 = hashlib.sha256()
hasher1.update(text1.encode())
hash1 = hasher1.hexdigest()

hasher2 = hashlib.sha256()
hasher2.update(text2.encode())
hash2 = hasher1.hexdigest()

hash1 == hash2

True

1 файл целиком состоит из части текста взятого из файла 2

Попробуем наивно использовать оператор `in` для проверки.

- выключен парсинг футеров и хедеров, с их парсингов появляются (мета?) символы конца страницы, которые дают результат `False`

In [6]:
text1 = Docx2Txt("./data/text-report/case_3_input_1.docx").get_text()
text2 = Docx2Txt("./data/text-report/case_3_input_2.docx").get_text()

text1 in text2

True

## Методы нахождения сходства между текстовыми документами


Приведенные ниже методы требуется рассматривать при перестановках / перефразировании / изменении объема текста, что будет основной проблемой при сравнении документов.

### Чистка текста 

Любой используемый метод требует подготовки данных: удаления стоп-слов, лемматизация, стемматизация, удаление лишних символов и пунктуации.

- <https://habr.com/ru/articles/738176/>, <https://github.com/rjrahul24/ai-with-python-series/blob/main/08.%20Natural%20Language%20Processing/Preprocessing%20Pipeline/NLP%20Preprocessing%20Pipeline%20.ipynb> - описание типичных методов чистки текста
- <https://spacy.io/usage/processing-pipelines> - препроцессинг теста с помощью `spacy`

In [7]:
%pip install spacy

import sys
!{sys.executable} -m spacy download en en_core_web_sm 

Note: you may need to restart the kernel to use updated packages.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import spacy

In [9]:
def spacy_normalize(text: str) -> str:
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
    stops = nlp.Defaults.stop_words

    text = re.sub(r'[^\w\s.]', '', text)
    text = re.sub('\s+', ' ', text)

    text = text.lower()

    text = nlp(text)

    lemmatized = list()

    for word in text:
        lemma = word.lemma_.strip()
        if lemma and lemma not in stops:
            lemmatized.append(lemma)
               
    return " ".join(lemmatized)

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/var/folders/8h/64vbk3hj78g_dnckx03z4v9r0000gn/T/ipykernel_59780/1338797335.py:6: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


### Традиционные методы нахождения сходства между документами

Сходство Жаккара. <https://ru.wikipedia.org/wiki/Коэффициент_Жаккара>

Сходство считаеся как отношение общего множества термов из текстов 1 и 2 к сумме уникальных термов в каждом тексте.

${K_{J}={\frac {c}{a+b-c}}}$

- $a$ - термы текста 1
- $b$ - термы текста 2
- $c$ - пересечение множеств термов документов 1 и 2


In [10]:
def jaccard_similarity(a: set, b: set) -> float:
    if 0 == len(a) == len(b):
        return 1

    return (len(a.intersection(b))) / (len(a.union(b)))

In [11]:
text1_normalized = spacy_normalize(text1)
text2_normalized = spacy_normalize(text2)

jaccard_similarity(
    set(text1_normalized.split()),
    set(text2_normalized.split()),
)

/Users/zuzuka28/Documents/pp/simrep/research/.venv/lib/python3.12/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


0.5

Алгоритм шинглов. w-shingling <https://ru.wikipedia.org/wiki/Алгоритм_шинглов>

Алгоритм шинглов представляет собой классический метод для проверки текста на плагиат. 

- текст канонизируется ( чистка от предлогов, стематизация, лемматизация )
- текст разбивается на шинглы, наборы из N последовательных термов
- считается сходство жаккара по шинглам

In [12]:
# разбиение на шинглы
def w_shingle(target: list, w: int = 1) -> list[set]:
    num_words = len(target)

    if w > num_words or w == 0:
        raise Exception('invalid shingle number')

    return [" ".join(target[i:i + w]) for i in range(len(target) - w + 1)]


In [13]:
text1_normalized = spacy_normalize(text1)
text2_normalized = spacy_normalize(text2)

shingle_size = 2

jaccard_similarity(
    set(w_shingle(text1_normalized.split(), shingle_size)),
    set(w_shingle(text2_normalized.split(), shingle_size)),
)

0.3333333333333333

Использование групп слов или групп символов (n-gram) - довольно распространенный метод, который можно использовать совместно с другими моделями

### Векторное сходство документов

Современные подходы нахождения сходства документов предлагают рассматривать документы как векторы в некотором векторном пространстве и сравнивать для нахождения сходства.

В качестве метрики близости векторов ( сходства ) используется косинусное расстояние между этими векторами.

Для приведения предложений в векторную форму используются нижеперечисленные методы ( и другие ). 

Bag of Words (BoW) - модель представления документов как набора из отношений { терм : частотность использования терма  в документе }.  

BoW используется для задач классификации, используя частотность использования в качестве метрики для классификации.

Можно использовать BoW как способ задания векторов для предложений. 

Нахождение какого-либо сходства между предложениями простого BoW может быть недостаточно, поскольку BoW упускает семантику и оценку важности слов. 

Ниже приведен пример использования BoW и нахождения косинусной схожести, в результате получается квадратная матрица с рассчетом схожести входных параметров.

#TODO: исследовать итеративное расширение мешка слов без переучивания на всем наборе данных

In [14]:
%pip install scikit-learn numpy pandas

  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl (11.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 6.6 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl (23.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
from sklearn.feature_extraction.text import CountVectorizer # BoW

text1_normalized = spacy_normalize(text1)
text2_normalized = spacy_normalize(text2)

texts = [text1_normalized, text2_normalized]

vectorizer = CountVectorizer()
vectorizer.fit(texts)

vectorizer.get_feature_names_out()

bow = vectorizer.transform(texts)
bow = bow.toarray()

df = pd.DataFrame(
   bow,
   columns=vectorizer.get_feature_names_out(),
)

cosine_similarity(df, df)


/Users/zuzuka28/Documents/pp/simrep/research/.venv/lib/python3.12/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

Term Frequency-Inverse Document Frequency (TF-IDF) <https://en.wikipedia.org/wiki/Tf–idf> - статистическая метрика для измерения важности слов в документах (тексте).

TF-IDF своего рода BoW с поправкой на важность терма в документах. 

$TFIDF(t, d) = TF(t, d) * IDF(t)$

Term Frequency - частота употребления терма в конкретном документе, изменяет значимость слова в контексте документа.

$TF(t, d) = \frac{f(t, d)}{N}$

- $f(t,d)$ - количество использований терма t в документе
- $N$ - количество термов в документе

Есть и другие способы считать коэффициент TF, нормализуя его (см. вики)

Inverse Document Frequency - частота использования терма во всех документах. чем чаще слово употребляется во всех документах, тем меньше у него значимость.

$IDF(t, D) = \log \left( \frac{N}{df(t)} \right)$

- $N$ - количество документов
- $df(t)$ - количество документов, в которых используется терм

Есть и другие способы считать коэффициент IDF, нормализуя его (см. вики)

TF-IDF для предложения представляет из себя вектор оценок, состоящий из рассчитанных значений TF-IDF для каждого терма.

Поскольку оба коэффициенты зависят от входных параметров, то получившиеся векторы можно нормализовать для устранения зависимости с размером входных параметров.


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF

text1_normalized = spacy_normalize(text1)
text2_normalized = spacy_normalize(text2)

texts = [text1_normalized, text2_normalized]

vectorizer = TfidfVectorizer()
vectorizer.fit(texts)

vectorizer.get_feature_names_out()

bow = vectorizer.transform(texts)
bow = bow.toarray()

df = pd.DataFrame(
   bow,
   columns=vectorizer.get_feature_names_out(),
)

cosine_similarity(df, df)

/Users/zuzuka28/Documents/pp/simrep/research/.venv/lib/python3.12/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


array([[1.        , 0.57973867],
       [0.57973867, 1.        ]])

Okapi BM25  - <https://ru.wikipedia.org/wiki/Okapi_BM25>

Okapi BM25 - метрика на основе TD-IDF с поправками результата на основе длины документа.

Более современный 

word2vec <https://en.wikipedia.org/wiki/Word2vec>

word2vec - нейронка, генерирующая репрезентацию документов в векторах с учетом рядом стоящих слов.
один раз натренировали нейронку для того, чтобы она выдавала качественные репрезентации текста в виде векторов, далее используем результат для других прикладных задач, требующих векторное представление текста.

вариации и развитие идеи

- sentence2vec
- doc2vec

BERT <https://en.wikipedia.org/wiki/BERT_(language_model)>

BERT - трансформер нейронка с кодировщиком, ее можно использовать для генерации векторной репрезентации текста.
BERT учитывает расположение и семантику слов в тексте.

<https://gist.github.com/albahnsen/b02d2183c93067e3f248a428430c970e> - про то, как достать sentence embeddings из BERT.

SBERT - более узкоспециализированная модель, предназначенная именно для поиска схожих документов

In [18]:
%pip install sentence_transformers

  Using cached sentence_transformers-3.1.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached torch-2.4.1-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached huggingface_hub-0.25.1-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-10.4.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached regex-2024.9.11-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached safetensors-0.4.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.20.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached mpmath-1.3.0-py3-none

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode([
    text1,
    text2
])

similarities = model.similarity(embeddings, embeddings)

similarities

/Users/zuzuka28/Documents/pp/simrep/research/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/zuzuka28/Documents/pp/simrep/research/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([[1.0000, 0.7684],
        [0.7684, 1.0000]])

## Методы нахождения сходства между документами из изображений

тест 1.1. идентичные файлы

In [20]:
images1 = Docx2Txt("./data/image-report/case_1_input_1.docx").get_images()
images2 = Docx2Txt("./data/image-report/case_1_input_2.docx").get_images()

images1 == images2

True

тест 1.2. файлы с разным форматированием и измененным размером изображений

In [21]:
images1 = Docx2Txt("./data/image-report/case_2_input_1.docx").get_images()
images2 = Docx2Txt("./data/image-report/case_2_input_2.docx").get_images()

hasher1 = hashlib.sha256()
hasher1.update(images1[0].data)
hash1 = hasher1.hexdigest()


hasher2 = hashlib.sha256()
hasher2.update(images2[0].data)
hash2 = hasher2.hexdigest()

hash1 == hash2

False

Перцептуальные хэши <https://en.wikipedia.org/wiki/Perceptual_hashing> и <https://en.wikipedia.org/wiki/Locality-sensitive_hashing> 

Перцептуальные хэши - алгоритм для "хэширования" изображения на основе набора признаков. Это не криптографические хэши, а скорее способ для фингерпринта (снятия образа данных).

Перцептуальные хэши можно сравнивать как напрямую, так и использовать для нахождения Расстояния Хэмминга между ними.

In [22]:
%pip install imagehash pillow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached ImageHash-4.3.1-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached pywavelets-1.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
Using cached pywavelets-1.7.0-cp312-cp312-macosx_11_0_arm64.whl (4.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import imagehash
import io
from PIL import Image

In [24]:
image1 = Image.open(io.BytesIO(images1[0].data))
image2 = Image.open(io.BytesIO(images2[0].data))

# <https://content-blockchain.org/research/testing-different-image-hash-functions/>
hashfuncs = [
    imagehash.phash,
    imagehash.average_hash,
    imagehash.whash,
    imagehash.dhash,
    imagehash.dhash_vertical,
]

threshold=6.4

for hashfunc in hashfuncs:
    h1 = hashfunc(image1)
    h2 = hashfunc(image2)

    print(f"\
        is eq {h1 == h2},\
        is eq with threshold {h1 - h2 < threshold},\
        hamming dist {h1 - h2},\
        hashlen {len(h1)}")
    

        is eq False,        is eq with threshold True,        hamming dist 2,        hashlen 64
        is eq False,        is eq with threshold True,        hamming dist 1,        hashlen 64
        is eq True,        is eq with threshold True,        hamming dist 0,        hashlen 64
        is eq False,        is eq with threshold True,        hamming dist 3,        hashlen 64
        is eq False,        is eq with threshold True,        hamming dist 1,        hashlen 64


CLIP и sentence transformers <https://huggingface.co/sentence-transformers/clip-ViT-B-32>

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("clip-ViT-B-32")
embeddings = model.encode([
    image1,
    image2
])

similarities = model.similarity(embeddings, embeddings)

similarities

tensor([[1.0000, 0.9986],
        [0.9986, 1.0000]])